In [447]:
import pandas as pd
import numpy as np
from scipy import sparse

# Introdução:

Esse documento tem como objetivo analisar o famoso algoritmo PageRank. Para isso serão respondidas algumas questões contextualizadas sobre bitcoin utilizando o PageRank para resolvê-las. A base de dados que será utilizada está disponivel em:https://snap.stanford.edu/data/soc-sign-bitcoinotc.csv.gz. A referência do PageRank utilizado está na bibliografia.

# Sobre os Dados:

Os dados consistem de um csv com quatro colunas:
<ul>
    <li>
        <b>SOURCE</b>: Usuário de origem 
    </li>
    <li>
        <b>TARGET</b>: Usuário de destino
    </li>
    <li>
        <b>RATING</b>: Nota atribuida pelo source ao target
    </li>
    <li>
        <b>TIME</b>: O tempo que ocorreu a atribuição da nota
    </li>
</ul>


# Preparando os dados:

Primeiramente iremos ler o csv supracitado:

In [448]:
df = pd.read_csv('soc-sign-bitcoinotc.csv')

Após isso, iremos filtrar os dados, visto que para nosso experimento somente nos interessa notas(rating) acima de 8

In [449]:
formattedDF = df.loc[df.RATING >= 8]

# Funções Auxiliares:

In [450]:
"""
    Função que retorna estruturas de dados necessárias para a execução do pageRank:
    links: Mapa de arrays que contem as informações de quem o usuário recebeu a nota. 
    numberOfLinks: Mapa de inteiros que contem as informações de quantas vezes o usuários atribuiu nota a alguem 
    pageRank: Inicia o mapa de pageRank com valores 1 inicias para todos os usuários.
"""
def getPageRankDataStructures(formattedDF):
    size = len(formattedDF["SOURCE"].values)
    pageRank = {}
    numberOfLinks = {}
    links = {}
    for index in range(size):
        src = formattedDF["SOURCE"].values[index]
        target = formattedDF["TARGET"].values[index]
        #initializing pageRank values
        pageRank[src] = 1.0

        if target not in links:
            links[target] = []
            links[target].append(src)
        else:
            links[target].append(src)

        if src not in numberOfLinks:
            numberOfLinks[src] = 1.0
        else:
            numberOfLinks[src] += 1.0
            
    return links, numberOfLinks, pageRank

In [453]:
# Faz a iteração do pageRanks
def doPageRankInteration(links, numberOfLinks, pageRank):
    # Necessário para que os valores das iterações não se confundam quando forem se alterando os valores
    # Em outras palavras, armazena o pageRank da iteração anterior
    pageRankCopy = {**pageRank}
    continueIteration = True
    countLoop = 0
    while (continueIteration):
        for user in pageRank:
            pageRank[user] = 0.15 + 0.85*sumPageRank(links, numberOfLinks, user, pageRankCopy)
        
        continueIteration = False
        
        for user in pageRank:
            value = pageRank[user] - pageRankCopy[user]

            if (value > 0.001):
                continueIteration = True
                break
                
        countLoop += 1
        pageRankCopy = {**pageRank}
    return pageRank, countLoop

In [454]:
# Calcula a soma relacionada a segunda parte da equação 
def sumPageRank(links, numberOfLinks, user, pageRankCopy):
    soma = 0.0
    if (user not in links):
        return soma
    
    for userLinked in links[user]:
        soma += (float(pageRankCopy[userLinked])/numberOfLinks[userLinked])
    
    return soma

# Variaveis auxiliares:

A partir das funções auxiliares descritas na seção anterior, criamos algumas váriaveis auxiliares que nos ajudarão a deixar o processo mais rápido:

In [455]:
#Armazena links, numberOfLinks e pageRank descritos acima
links, numberOfLinks, pageRank = getPageRankDataStructures(formattedDF)

In [458]:
soma = 0
for userLinked in links[1]:
        soma += (float(pageRank[userLinked])/numberOfLinks[userLinked])

In [460]:
#Armazena o valor final do pageRank e o número de loops
iteredPageRank, countLoop = doPageRankInteration(links, numberOfLinks, pageRank)


# Perguntas

# Quantas iterações o PageRank precisou rodar até atingir convergência?

O número de iterações necessários foi de 43. O limiar escolhido foi quando o valor da iteração n-1 e n de todos os usuários não ultrapassou a diferença de 0.001.

# Quais os 5 investidores mais importantes segundo o PageRank? Quais seus valores de PageRank?

In [ ]:
#Ordena os usuários por valor de pageRank
orderedPageRank = [(key, iteredPageRank[key]) for key in sorted(iteredPageRank, key=iteredPageRank.get, reverse=True)]

<table style="width:100%">
    <tr>
        <th> ID Usuário </th>
        <th> Page Rank</th>

    </tr>
    <tr>
        <td> 1 </td>
        <td> 8.62530181044244 </td>
    </tr>
    <tr>
        <td> 4172 </td>
        <td> 4.195103781381812 </td>
    </tr>
    <tr>
        <td> 2642 </td>
        <td> 4.126206533967323 </td>

    </tr>
    <tr>
        <td> 1018 </td>
        <td> 3.240076144653836 </td>
    </tr>
     <tr>
        <td> 2647 </td>
        <td> 2.9295767398264307 </td>
    </tr>
</table>

# Como você poderia usar o PageRank caso você fosse um investidor em bitcoins?
    
 
    


   <div style="text-align: justify;"> 
        Considerando o experimento, acredito que um bom jeito de utilizar o PageRank seria para produzir um indice de verificador de confiança. No inicio do experimento podemos ver que há vários usuários com várias transações maiores que 8, contudo, o page rank pega este escopo(do experimento) e determina os usuários mais relevantes, gerando assim uma nova métrica de confiabilidade de usuários para transações. 
        
    </div>

# Bibliografia

http://pr.efactory.de/e-pagerank-algorithm.shtml